In [1]:
from google.colab import files
files.upload()

Saving cats_and_dogs_small.zip to cats_and_dogs_small.zip


In [3]:
from zipfile import ZipFile
file_name = "cats_and_dogs_small.zip"

with ZipFile(file_name, "r") as zip:
  zip.extractall()
  print("Done")

Done


In [4]:
IMAGE_SIZE = [150,150]

train_path = "/content/train"
valid_path = "/content/test"

In [6]:
#Data Augmentation

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                   width_shift_range=0.2, 
                                   height_shift_range=0.2,
                                   shear_range=0.2, zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [11]:
train_set = train_datagen.flow_from_directory(
                                              train_path,
                                              target_size=(150,150),
                                              batch_size=20,
                                              class_mode="binary"
                                              )
 
test_set = test_datagen.flow_from_directory(
                                              valid_path,
                                              target_size=(150,150),
                                              batch_size=20,
                                              class_mode="binary"
                                              )

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [14]:
from tensorflow.keras import models, Sequential, optimizers
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv2D, MaxPooling2D

In [24]:
model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation="relu", input_shape=(150, 150, 3), padding="same"))
model.add(layers.MaxPooling2D(2,2))

model.add(layers.Conv2D(64, (3, 3), activation="relu", padding="same"))
model.add(layers.MaxPooling2D(2,2))

model.add(layers.Conv2D(128, (3, 3), activation="relu", padding="same"))
model.add(layers.MaxPooling2D(2,2))

model.add(layers.Flatten())

model.add(layers.Dense(512, activation="relu"))

model.add(layers.Dense(1, activation="sigmoid"))

In [30]:
model.compile(
  loss='binary_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [31]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 150, 150, 32)      896       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 75, 75, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 75, 75, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 37, 37, 64)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 37, 37, 128)       73856     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 18, 18, 128)       0         
_________________________________________________________________
flatten (Flatten)            (None, 41472)            

In [33]:
r = model.fit_generator(
                        train_set,
                        validation_data=test_set,
                        epochs = 5,
                        steps_per_epoch=len(train_set),
                        validation_steps=len(test_set)
                        )

Epoch 1/5
100/100 [==============================] - 126s 1s/step - loss: 0.6545 - accuracy: 0.6380 - val_loss: 0.6279 - val_accuracy: 0.6570
Epoch 2/5
100/100 [==============================] - 127s 1s/step - loss: 0.6453 - accuracy: 0.6210 - val_loss: 0.6226 - val_accuracy: 0.6560
Epoch 3/5
100/100 [==============================] - 124s 1s/step - loss: 0.6313 - accuracy: 0.6585 - val_loss: 0.6014 - val_accuracy: 0.6840
Epoch 4/5
100/100 [==============================] - 124s 1s/step - loss: 0.5995 - accuracy: 0.6685 - val_loss: 0.6195 - val_accuracy: 0.6710
Epoch 5/5
100/100 [==============================] - 124s 1s/step - loss: 0.6000 - accuracy: 0.6665 - val_loss: 0.6075 - val_accuracy: 0.6700


In [34]:
model.predict(test_set)[2]

array([0.3046569], dtype=float32)